![image.png](https://github.com/descendant-ai/functime/raw/main/docs/img/banner_dark_bg.png)

[functime](functime.ai) is a lightweight time series library that enables scaling forecasts of large panel datasets effortless, thanks to Polars and global forecasting.

In other words: with functime, you can **train forecasting models on thousands of time series on your laptop** (no PySpark required!), and feature engineering is blazingly fast thanks to Polars' parralelisation capabilities.

# What you'll see

1. How functime supports you in displaying panel datasets.
2. How functions natively operate on panel datasets, abstracting away the aggregations.
3. Provides naive models to benchmark your forecasts.
4. Has a sklearn-compatible API.
5. Makes it easy to perform time-series cross-validation.
6. Provides utilities to rank your forecasts and residuals to improve model diagnostics.
7. Provides plotting utilities to spot poorly-behaved time series in the panel.

In [ ]:
try:
    import functime
except ModuleNotFoundError:
    # NOTE: this might fail on Kaggle: it requires an account and phone verification.
    # If it happens, go to the "sidebar > Notebook options" and verify your phone number.
    !pip install --quiet -- "functime[plot,lgb]"

import polars as pl
import functime

print(
    f"polars: {pl.__version__}",
    f"functime: {functime.__version__}",
    sep="\n",
)

# 1. Data Description and Problem Statement

The dataset contains monthly observations of some 70 commodities, between 1960 and 2023.

In [ ]:
url = "https://github.com/neocortexdb/functime/raw/main/data/commodities.parquet"
y = pl.scan_parquet(url).with_columns(pl.col("time").cast(pl.Date))

y.group_by("commodity_type").tail().collect()

A panel dataset always contains a column that identifies the time series, the datetime indexes, and the target values. There can also be additional covariates, such as holiday effects or macroeconomic variables.

functime assumes that the first column is always the time series identifier (in the documentation, it is referred to as _entity column_), and is followed by the time column and the target column.

> RFC: Do you like these defaults? What do you think of a `PanelDataFrame` class that inherits from a `polars.DataFrame`, whose constructor allows you to specify the names of the entity, time and target cols?

In [ ]:
entity_col, time_col, target_col = y.columns

The data has a monthly frequency. Our goal is to forecast the price of the next quarter.

In [ ]:
freq = "1mo"
forecasting_horizon = 4 # a quarter

# 2. Exploratory Data Analysis

The number of observations per entity is not constant: as we can see from the plot below, the number ranges from approximately 250 to 750.

In [ ]:
from functime import plotting

plotting.plot_entities(y)

How do these series look like? Since there are 70 series (or thousands, potentially), we cannot inspect each of them. We could sample some:

In [ ]:
# Here is a little bit of magic to display a sample of time series.
# You won't have to do this in functime 0.10.0 coming out December 2023!

k = 6
seed = 42

commodities = (
    y
    .select(pl.col(entity_col).unique())
)

# sample K entity IDs to plot
sample = (
    commodities
    .collect()
    .to_series()
    .sample(k, seed=seed)
)

(
    y
    .filter(pl.col(entity_col).is_in(sample))
    .pipe(
        plotting.plot_panel,
        title=f"Sample of {k} time series",
        height=125*k,
    )
)

> In the next release, there will be a `ids: int | Container[int] | str | Container[str]` parameter to plot specific entities; there will also be a `shuffle` parameter to sample `ids` if it is an int.

For this reason, we should look at other time series summary statistics. One of such is the coefficient of variation, measured as the variance of a series divided by its mean:

In [ ]:
# With the next release, you will also be able to show the top/bottom K 
# series in the panel ranked according a scoring function

def coefficient_of_variation(col):
    return pl.col(target_col).std() / pl.col(target_col).mean() 

top_k_cv = (
    y
    .group_by(entity_col)
    .agg(coefficient_of_variation(time_col).alias("cv"))
    .top_k(k=k, by="cv")
    .collect()
    .get_column(entity_col)
)

(
    y
    .filter(pl.col(entity_col).is_in(top_k_cv))
    .pipe(
        plotting.plot_panel,
        title=f"Top {k} series by coefficient of variation",
        height=125*k,
    )
)

# 3. Train-test split

functime features the familiar scikit-learn API:

In [ ]:
from functime.cross_validation import train_test_split

splitter = train_test_split(forecasting_horizon)
y_train, y_test = splitter(y)

functime functions, however, are designed to work with panel data:

In [ ]:
y_test.head(12).collect()

# 4. Benchmark forecast

Some forecasting methods are extremely simple and surprisingly effective. Naive and seasonal naive forecasters are surprisingly hard to beat! You should always consider using the naive and seasonal naive forecasts as benchmarks. The naive forecaster repeats the last value and, for a multi-step forecast, returns a flat line. The seasonal naive returns the same pattern of length `sp` (seasonal periodicity). [reference](https://otexts.com/fpp3/simple-methods.html#simple-methods)

In [ ]:
from functime.forecasting import snaive

forecaster_naive = snaive(
    freq=freq,
    sp=12,
)

Now we train the seasonal naive model:

In [ ]:
_ = forecaster_naive.fit(y=y_train)
y_bench_pred = forecaster_naive.predict(fh=forecasting_horizon)

y_bench_pred.head(12)

## 4.1 Diagnostics

In [ ]:
plotting.plot_forecasts(
    y_test.filter(pl.col(entity_col).is_in(sample)).collect(),
    y_bench_pred.filter(pl.col(entity_col).is_in(sample)),
    title=f"Sample of {k} naive forecasts",
)

Once again, how can we evaluate the goodness of fit across dozens of series? functime offers the `rank_*` functions:

In [ ]:
from functime.evaluation import rank_point_forecasts

rank_bench_point_forecast = rank_point_forecasts(
    y_true=y_test,
    y_pred=y_bench_pred,
    sort_by="mape",
    descending=True,
)

rank_bench_point_forecast.head()

In [ ]:
worst_bench_point_forecast = rank_bench_point_forecast.head(k).get_column(entity_col)

plotting.plot_forecasts(
    y_test.filter(pl.col(entity_col).is_in(worst_bench_point_forecast)).collect(),
    y_bench_pred.filter(pl.col(entity_col).is_in(worst_bench_point_forecast)),
    title=f"Worst {k} naive forecasts by mean absolute percentage error (MAPE)",
    height=125*k,
)

There are, however, better approaches to glance the goodness of fit - which we shall see after we fit another model.

# 5. Forecasting with global models

What makes functime so fast is also the fact that only one model is fit on the whole dataset - hence why the name _global forecasting_. This strategy first proved successful with the [M5 competition](https://www.sciencedirect.com/science/article/pii/S0169207021001874?via%3Dihub), while there is a growing amount of [literature](https://www.sciencedirect.com/science/article/abs/pii/S0169207021000558?via%3Dihub) (mostly from Hyndman and his coauthors). This enables a ["new paradigm of forecasting"](https://blogs.sas.com/content/forecasting/2016/10/25/changing-the-paradigm-for-business-forecasting-part-10/):

> [...] [T]he amount of time, effort, and resources spent on forecasting is not commensurate with the benefit achieved – the improvement in accuracy.

> We spend far too many resources generating, reviewing, adjusting, and approving our forecasts, while almost invariably failing to achieve the level of accuracy desired. The evidence now shows that a large proportion of typical business forecasting efforts fail to improve the forecast, or even make it worse. So the conversation needs to change. The focus needs to change.

> We need to shift our attention from esoteric model building to the forecasting process itself – its efficiency and its effectiveness.

## 5.1 Linear models

A simple linear regression can be made a global forecasting model simply by fitting in on the whole dataset.

In [ ]:
from functime.forecasting import linear_model
from functime.preprocessing import scale, add_fourier_terms, roll
from functime.seasonality import add_calendar_effects

target_transforms = scale()
feature_transforms = roll(window_sizes=(6, 12, 24), stats=("mean", "std"), freq=freq)

forecaster_linear = linear_model(
    freq=freq,
    lags=12,
    target_transform=target_transforms,
    feature_transform=feature_transforms,
)

In this case, we scale the target variable and perform a couple of other common steps in feature engineering for time series:

1. Add 12 lagged values of the target variable.
2. Compute the rolling mean and standard deviation with window size of 6 and 12 and 24 (6 total).

functime also supports de-trending and de-seasonalisation, boxcox transform but also fractional differential transform as explained in the influential Advances in Financial Machine Learning by Marcos Lopez de Prado. See them all [here](https://docs.functime.ai/ref/preprocessing).

To improve our model evaluation, we also perform time-series cross validation with 5 splits of length 180 (i.e., 15 years worth of data).

In [ ]:
backtesting_opts = dict(
    y=y_train,
    window_size=180, # 15 years of training data in each fold
    test_size=forecasting_horizon,
    step_size=1,
    n_splits=5
)

In [ ]:
%%time
y_linear_preds, y_linear_resids = forecaster_linear.backtest(**backtesting_opts)

### 5.1.1 Diagnostics

In [ ]:
plotting.plot_backtests(
    y_train.filter(pl.col(entity_col).is_in(sample)).collect(),
    y_linear_preds.filter(pl.col(entity_col).is_in(sample)),
    title=f"Sample of {k} cross-validated linear forecasts",
    last_n=36,
    height=125*k
)

We can even compare residuals:

In [ ]:
from functime.evaluation import rank_residuals

rank_linear_residuals = rank_residuals(
    y_resids=y_linear_resids,
    sort_by="abs_bias",
    descending=True
)

best_linear_residuals = rank_linear_residuals.tail(k).get_column(entity_col)

plotting.plot_residuals(
    y_resids=y_linear_resids.filter(pl.col(entity_col).is_in(best_linear_residuals)),
    n_bins=200,
    height=800,
    width=1000,
    title=f"Top {k} linear forecast residuals",
)

## 5.2 Gradient Boosted Trees

In [ ]:
from functime.forecasting import lightgbm

forecaster_gbm = lightgbm(
    freq=freq,
    lags=12,
    target_transform=target_transforms,
    feature_transform=feature_transforms
)

In [ ]:
%%time
y_lgb_preds, y_lgb_resids = forecaster_gbm.backtest(**backtesting_opts)

### 5.1.2 Diagnostics

In [ ]:
plotting.plot_backtests(
    y_train.filter(pl.col(entity_col).is_in(sample)).collect(),
    y_lgb_preds.filter(pl.col(entity_col).is_in(sample)),
    title=f"Sample of {k} cross-validated LightGBM forecasts",
    last_n=36,
    height=125*k
)

In [ ]:
rank_lgb_residuals = rank_residuals(
    y_resids=y_lgb_resids,
    sort_by="abs_bias",
    descending=True
)

best_lgb_residuals = rank_lgb_residuals.tail(k).get_column(entity_col)

plotting.plot_residuals(
    y_resids=y_lgb_resids.filter(pl.col(entity_col).is_in(best_lgb_residuals)),
    n_bins=200,
    height=800,
    width=1000,
    title=f"Top {k} LightGBM forecast residuals",
)

# 6. Comparing different models

Currently naive forecasters raise an error when `.backtest` is called on them. The error is caused by computing the residuals; it will be fixed before the next minor release.

In [ ]:
from functime.backtesting import backtest
from functime.cross_validation import sliding_window_split, train_test_split

splitter = train_test_split(forecasting_horizon)
y_train, y_test = splitter(y)

# make a new forecast each month
step_size = 1

# backtest: refit one year
n_splits = 12

cv_sliding = sliding_window_split(
    test_size=backtesting_opts["test_size"],
    step_size=backtesting_opts["step_size"],
    n_splits=backtesting_opts["n_splits"],
    window_size=backtesting_opts["window_size"],
)

y_bench_preds = backtest(
    forecaster=forecaster_naive,
    fh=backtesting_opts["test_size"],
    y=backtesting_opts["y"],
    cv=cv_sliding,
    residualize=False # currently unsupported, will be fixed in time for 0.10.0
)

The comet plot uses sMAPE (currently, will be fixed):

$\text{sMAPE} = \text{mean}\left(200|y_{t} - \hat{y}_{t}|/(y_{t}+\hat{y}_{t})\right)$

This metric has its weaknesses, despite being used (e.g. in M3):

> However, if $y_t$ is close to zero, $\hat{y}_{t}$ is also likely to be close to zero. Thus, the measure still involves division by a number close to zero, making the calculation unstable. Also, the value of sMAPE can be negative, so it is not really a measure of “absolute percentage errors” at all. [ref](https://otexts.com/fpp3/accuracy.html#percentage-errors)

## 6.1 Forecast Value Added Plot (FVA)

Compares the performance of a foreaster's prediction against a naive foreaster, for all time series in the panel.

In [ ]:
plotting.plot_fva(
    y_train,
    y_lgb_preds,
    y_bench_preds,
    height=900,
    width=900,
    title="SMAPE: Seasonal Naive (x axis) and LightGBM (y axis)",
)

In [ ]:
plotting.plot_fva(
    y_train,
    y_linear_preds,
    y_bench_preds,
    height=900,
    width=900,
    title="SMAPE: Seasonal Naive (x axis) and Linear (y axis)",
)

## 6.2 Comet Plot

Plot the forecast errors against the coefficient of variation

In [ ]:
plotting.plot_comet(
    y_train=y_train.collect(),
    y_test=y_train.collect(),
    y_pred=y_lgb_preds,
    height=900,
    width=900,
    title="LightGBM: Coefficient of Variation (x) vs SMAPE (y)",
)

In [ ]:
plotting.plot_comet(
    y_train=y_train.collect(),
    y_test=y_train.collect(),
    y_pred=y_linear_preds,
    height=900,
    width=900,
    title="Linear Model: Coefficient of Variation (x) vs SMAPE (y)",
)